Search for granules on the cloud for a box of interest, run custom candidate fire pixel extraction, map I4 + custom candidates.

In [11]:
#conda activate /projects/myenvs/candidates-env
import xarray as xr
import earthaccess
from earthaccess import Auth, Store, DataCollections, DataGranules
import datetime as dt
import time
import os
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import folium
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import rioxarray
from scipy.spatial.distance import cdist

In [12]:
output_dir = '/projects/shared-buckets/qulizad/scripts/outputs/canada/'

In [13]:
def make_dir(fire_name):
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

    if not os.path.isdir(output_dir + fire_name):
        os.mkdir(output_dir + fire_name)

In [14]:
def fetch_data(satellite):
    
    if satellite=='SNPP': products = ['VNP03IMG','VNP02IMG','VNP14IMG']
    elif satellite=='NOAA20': products = ['VJ103IMG','VJ102IMG','VJ114IMG']
    
    s3_links = {}
    files = {}
    for product in products:
        #query for granules - by bounding box or point
        Query = DataGranules().short_name(product).bounding_box(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]).temporal(START,END)

        cloud_granules = Query.get(800) #first 800 results

        #save granule URLs to list
        s3_links[product] = []
        for granule in cloud_granules:
            s3_links[product].extend(granule.data_links(access="direct"))
        s3_links[product] = sorted(s3_links[product]) 
        files[product] = store.open(s3_links[product], provider="LAADS")

    return s3_links, files

In [15]:
#first get the L1 geolocation and science data via direct S3 access
#may require restarting the kernel if permission errors

files = {}

#geolocation 03IMG
results = earthaccess.search_data(
    short_name=products[0],
    bounding_box=(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]),
    temporal=(START, END),
    count=10
)
files[products[0]] = earthaccess.open(results)

#science data 02IMG
results = earthaccess.search_data(
    short_name=products[1],
    bounding_box=(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]),
    temporal=(START, END),
    count=10
)
files[products[1]] = earthaccess.open(results)

files

Granules found: 19
Opening 10 granules, approx size: 1.65 GB
using provider: LAADS


QUEUEING TASKS | : 10it [00:00, 818.50it/s]
PROCESSING TASKS | : 100%|██████████| 10/10 [00:00<00:00, 16.00it/s]
COLLECTING RESULTS | : 100%|██████████| 10/10 [00:00<00:00, 134432.82it/s]


Granules found: 19
Opening 10 granules, approx size: 1.72 GB
using provider: LAADS


QUEUEING TASKS | : 10it [00:00, 2051.41it/s]
PROCESSING TASKS | : 100%|██████████| 10/10 [00:00<00:00, 468.00it/s]
COLLECTING RESULTS | : 100%|██████████| 10/10 [00:00<00:00, 135300.13it/s]


{'VNP03IMG': [<File-like object S3FileSystem, prod-lads/VNP03IMG/VNP03IMG.A2023122.0848.002.2023122170637.nc>,
  <File-like object S3FileSystem, prod-lads/VNP03IMG/VNP03IMG.A2023122.1030.002.2023122184811.nc>,
  <File-like object S3FileSystem, prod-lads/VNP03IMG/VNP03IMG.A2023122.1836.002.2023123012302.nc>,
  <File-like object S3FileSystem, prod-lads/VNP03IMG/VNP03IMG.A2023122.2018.002.2023123044510.nc>,
  <File-like object S3FileSystem, prod-lads/VNP03IMG/VNP03IMG.A2023123.0830.002.2023123165012.nc>,
  <File-like object S3FileSystem, prod-lads/VNP03IMG/VNP03IMG.A2023123.1006.002.2023123182615.nc>,
  <File-like object S3FileSystem, prod-lads/VNP03IMG/VNP03IMG.A2023123.1818.002.2023124031745.nc>,
  <File-like object S3FileSystem, prod-lads/VNP03IMG/VNP03IMG.A2023123.2000.002.2023124050027.nc>,
  <File-like object S3FileSystem, prod-lads/VNP03IMG/VNP03IMG.A2023124.0806.002.2023124163101.nc>,
  <File-like object S3FileSystem, prod-lads/VNP03IMG/VNP03IMG.A2023124.0948.002.2023124163229.nc>

In [16]:
def run_fire_algorithm():
    all_dets = pd.DataFrame()
    
    products = list(s3_links.keys())

    for i in range(len(products[0])): #VNP02IMG or VJ102IMG
        timestamp = s3_links[products[0]][i].split('.')[1:3]
        print(timestamp)
        year = timestamp[0][1:5]
        day = timestamp[0][5:8]
        time = timestamp[1]
        date = dt.datetime.strptime(year+day, '%Y%j').strftime('%b %d') 
        acq_datetime = dt.datetime.strptime(year+day+time[:2]+time[2:], '%Y%j%H%M').strftime('%Y-%m-%d %H:%M:00 +00:00') 
        daytime = int(time) > 1500 #depends on timezone
        
        try:
            #open VNP03IMG geolocation
            geo = xr.open_dataset(files[products[1]][i], engine='h5netcdf', group='geolocation_data')
            lon = geo['longitude'][:]
            lat = geo['latitude'][:]

            scene = (lon > EXTENT[0]) & (lon < EXTENT[2]) & (lat > EXTENT[1]) & (lat < EXTENT[3])

            #crop down the datasets for memory 
            indices = np.where(scene)
            x0 = indices[0].min()
            x1 = indices[0].max()
            y0 = indices[1].min()
            y1 = indices[1].max()

            lon = lon[x0:x1, y0:y1]
            lat = lat[x0:x1, y0:y1]

            #get VNP02IMG science data, i1-i5 bands
            data = xr.open_dataset(files[products[0]][i], engine='h5netcdf', group='observation_data')
            data = data.sel(number_of_lines=slice(x0,x1), number_of_pixels=slice(y0,y1))

            i4 = data['I04'] #xarray already encodes the scale factor and offset
            scale = data.I04.encoding['scale_factor']
            offset = data.I04.encoding['add_offset']
            i4 = (i4[:,:] - offset) / scale #return to raw values to use lookup table to temperature
            i4 = i4.astype(int)
            i4_bt = data['I04_brightness_temperature_lut'][:]
            i4_bt = i4_bt[i4]

            i5 = data['I05']
            scale = data.I05.encoding['scale_factor']
            offset = data.I05.encoding['add_offset']
            i5 = (i5[:,:] - offset) / scale
            i5 = i5.astype(int)
            i5_bt = data['I05_brightness_temperature_lut'][:]
            i5_bt = i5_bt[i5]

        except:
            print('error with file or does not exist',timestamp)
            continue


        if daytime:
            clouds = (i5_bt < 265) | ((data.I01+data.I02 > 0.9) & (i5_bt < 295)) | ((data.I01+data.I02 > 0.7) & (i5_bt < 285)) 
            water = (data.I01 > data.I02) & (data.I02 > data.I03)
            saturated = (i4_bt==367) & (data.I04_quality_flags==9) & (data.I05_quality_flags==0) & (i5_bt > 290) & (data.I01+data.I02 < 0.7)
            folded = (i4_bt-i5_bt < 0) & (i5_bt > 325) & (data.I05_quality_flags==0)
            bg_fires = ((i4_bt > 335) & (i4_bt-i5_bt > 30)) | (folded)
            bright = (data.I03 > 0.3) & (data.I03 > data.I02) & (data.I02 > 0.25) & (i4_bt <335)
            #glint = come back to this
            candidates = (~bright) & (i4_bt > 325) & (i4_bt-i5_bt > 25) #double check this isn't contextual

            fires =  (saturated | folded | bg_fires | candidates) #~clouds & ~water &

        else: #nighttime
            clouds = (i5_bt < 265) & (i4_bt < 295)
            unequivocal = (i4_bt > 320) & (data.I04_quality_flags==0)
            saturated = (i4_bt==367) & (data.I04_quality_flags==9) & (data.I05_quality_flags==0)
            folded = ((i4_bt-i5_bt < 0) & (i5_bt > 310) & (data.I05_quality_flags==0)) | ((i4_bt > 208) & (i5_bt > 335))
            bg_fires = ((i4_bt > 300) & (i4_bt-i5_bt > 10)) | (folded)
            #bright = water #all false
            candidates = (i4_bt > 295) & (i4_bt-i5_bt > 10) 

            fires = (unequivocal | saturated | folded | bg_fires | candidates) #~clouds & 


        #build pandas table for exporting, following VIIRS L2 columns
        i_dets = pd.DataFrame() #copy of master table just for this swath
        i_dets['longitude'] = list(np.array(lon)[fires])
        i_dets['latitude'] = list(np.array(lat)[fires])
        i_dets['acq_date'] = dt.datetime.strptime(year+day, '%Y%j').strftime('%Y/%m/%d') 
        i_dets['acq_time'] = time
        i_dets['acq_datetime'] = acq_datetime

        #crop down to defined extent
        i_dets = i_dets[(i_dets.longitude > EXTENT[0]) & (i_dets.longitude < EXTENT[2]) & (i_dets.latitude > EXTENT[1]) & (i_dets.latitude < EXTENT[3])]

        #FIGURE ----------------
        fig = plt.figure(figsize=(11,8))

        ax = fig.add_subplot(121, projection=ccrs.Miller())
        ax.set_extent([EXTENT[0],EXTENT[2],EXTENT[1],EXTENT[3]])
        plot = ax.pcolormesh(lon, lat, i4_bt, vmin=250, vmax=360, cmap='plasma', transform=ccrs.PlateCarree())
        cbar = plt.colorbar(plot, orientation='horizontal', shrink=0.6, pad=0.01, extend='both', ax=ax)
        cbar.ax.tick_params(labelsize=13)
        cbar.set_label('I4 brightness temperature (K)', size=13)
        ax.set_title(f'{s} {date} {time}h UTC') 
             
        ax2 = fig.add_subplot(122, projection=ccrs.Miller())
        ax2.set_extent([EXTENT[0],EXTENT[2],EXTENT[1],EXTENT[3]])
        plot = ax2.pcolormesh(lon, lat, i4_bt, vmin=250, vmax=360, cmap='plasma', transform=ccrs.PlateCarree())
        cbar = plt.colorbar(plot, orientation='horizontal', shrink=0.6, pad=0.01, extend='both', ax=ax2)
        cbar.ax.tick_params(labelsize=13)
        cbar.set_label('I4 brightness temperature (K)', size=13)
        ax2.scatter(i_dets.longitude, i_dets.latitude, c='1', s=1, transform=ccrs.Geodetic())
        ax2.text(0.35, 0.9, 'Potential fire pixels', c='white', transform = ax2.transAxes)
        figimgs = (f'{output_dir}/{fire_name}/{timestamp[0]}-{timestamp[1]}_{s}.png')
        plt.savefig(figimgs, dpi=100)
        plt.close()
        all_dets = pd.concat([all_dets, i_dets])
    
    #save csv with filename as the timestamp range
    filecsv = (f'{output_dir}/{timestamp[0]}_{s}.csv')
    all_dets.to_csv(filecsv, index=False)

    print('done')

Main

In [17]:
#start = time.time()
#start

In [18]:
#read in pyroCB catalog
file = f'/projects/shared-buckets/qulizad/NRL_pyroCb_inventory_2023_v1.csv'
df = pd.read_csv(file, header=0, usecols=["datetime", "fire_name", "region", "lat", "lon"])
rslt_df = df.loc[(df['region'] == 'Canada')]  #.reset_index(drop=True) #subsetting to Canada


In [19]:
#rslt_df.loc[25] ## only prints up to not including 10th record

In [20]:
#initiate cloud session - need to reauthenticate every hour :(

auth = Auth() 
#auth.login(strategy="interactive", persist=True) #RUN THIS THE FIRST TIME
auth.login(strategy="netrc") #read credentials from previously saved ~/.netrc file

store = Store(auth)
fs = store.get_s3fs_session('LAADS') #daac or provider name

In [21]:
#loop through all pyroCBs, run methods to make output directories, fetch data, and generate candidate fires

#%time
# %time is not the same as %%time because the former only see's how long the current 
#line takes to execute, whereas the latter checks the how the current line and 
#following lines take to execute
#[:] is the array slice syntax for every element in the array

#start time 

for i in rslt_df.index[1:2]:
    #if i%5==0: #reauthenticate
    
    #current & elapsed = ...
    #if elapsed > 0.98: #reauthenticate, reset "start" time
        
    fire_name = rslt_df.loc[i, 'fire_name']
    lat = rslt_df.loc[i, 'lat']
    lon = rslt_df.loc[i, 'lon']
    dat = rslt_df.loc[i, 'datetime']
    date_object = dt.datetime.strptime(dat, "%m/%d/%y %H:%M")    
    print(i, fire_name, date_object)
    
    start_date = date_object + dt.timedelta(-2)
    end_date = date_object + dt.timedelta(+2)
    
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    
    make_dir(fire_name)

    EXTENT = [lon - 0.5, lat - 0.5, lon + 0.5, lat + 0.5]
    START = start_date
    END = end_date
    
    satellites = ['SNPP','NOAA20']
    for s in satellites:
        s3_links, files = fetch_data(s)
        
        if s=='SNPP': products = ['VNP03IMG','VNP02IMG','VNP14IMG']
        elif s=='NOAA20': products = ['VJ103IMG','VJ102IMG','VJ114IMG']
        
        run_fire_algorithm() #using level 1 data

        #run_custom_fire_algorithm(s3_links, files) #using level 1 data
        #run_viirs_l2_fire_algoithm(s3_links, files) #using level 2 data
        



3 23BN-VERMETTE 2023-05-04 23:10:00


QUEUEING TASKS | : 19it [00:00, 1747.28it/s]
PROCESSING TASKS | : 100%|██████████| 19/19 [00:00<00:00, 20.34it/s]
COLLECTING RESULTS | : 100%|██████████| 19/19 [00:00<00:00, 113682.99it/s]
QUEUEING TASKS | : 19it [00:00, 2640.46it/s]
PROCESSING TASKS | : 100%|██████████| 19/19 [00:00<00:00, 475.62it/s]
COLLECTING RESULTS | : 100%|██████████| 19/19 [00:00<00:00, 247489.99it/s]
QUEUEING TASKS | : 19it [00:00, 3396.05it/s]
PROCESSING TASKS | : 100%|██████████| 19/19 [00:00<00:00, 79.19it/s]
COLLECTING RESULTS | : 100%|██████████| 19/19 [00:00<00:00, 187510.06it/s]


['A2023122', '0848']
error with file or does not exist ['A2023122', '0848']
['A2023122', '1030']
error with file or does not exist ['A2023122', '1030']
['A2023122', '1836']
error with file or does not exist ['A2023122', '1836']
['A2023122', '2018']
error with file or does not exist ['A2023122', '2018']
['A2023123', '0830']
error with file or does not exist ['A2023123', '0830']
['A2023123', '1006']
error with file or does not exist ['A2023123', '1006']
['A2023123', '1818']
error with file or does not exist ['A2023123', '1818']
['A2023123', '2000']
error with file or does not exist ['A2023123', '2000']


OSError: [Errno 30] Read-only file system: '/projects/shared-buckets/qulizad/scripts/outputs/canada//A2023123_SNPP.csv'